In [1]:
import configparser
import os
from joblib import dump, load
import json
from tqdm import tqdm
from helpers.helper_functions import *
from helpers.helper_classes import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from joblib import dump, load

pd.set_option('display.max_rows', 500)

c:\Users\caspa\Desktop\data-mining-techniques-vu\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
# Read config.ini file
config = configparser.ConfigParser()
config.read('src/config.ini')
# os.chdir(config['PATH']['ROOT_DIR'])

# # Load data
df = pd.read_csv(config['PATH']['DATA_DIR'] + '/training_set.csv')
# df

# mini df for testing quickly 
df_mini = df.iloc[:1000, :].copy()
# df.drop('Unnamed: 0', axis=1, inplace=True)

# time to datetime
# df['time'] = pd.to_datetime(df['time'])


In [53]:
def construct_target(row):
    if row['booking_bool'] == 1:
        return 1
    elif row['click_bool'] == 1:
        return 0.2
    else:
        return 0

def binary_target(row):
    if row['booking_bool'] == 1:
        return 1
    else:
        return 0

def drop_cols(df, cols):
    return df.drop(cols, axis=1)

In [54]:
## Test: Drop all columns with NaNs
df_1 = df
df_1 = df_1.dropna(axis=1)
df_1['target'] = df_1.apply(binary_target, axis=1)
df_1 = df_1.drop(['random_bool', 'click_bool', 'booking_bool', 'date_time'], axis=1)
df_1

C:\Users\caspa\AppData\Local\Temp\ipykernel_11256\4214444667.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['target'] = df_1.apply(binary_target, axis=1)


,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_brand_bool,prop_location_score1,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,target
0,1,12,187,219,893,3,1,2.83,4.95,27,104.77,0,23246,1,0,4,0,1,1,0
1,1,12,187,219,10404,4,1,2.20,5.03,26,170.74,0,23246,1,0,4,0,1,1,0
2,1,12,187,219,21315,3,1,2.20,4.92,21,179.80,0,23246,1,0,4,0,1,1,0
3,1,12,187,219,27348,2,1,2.83,4.39,34,602.77,0,23246,1,0,4,0,1,1,0
4,1,12,187,219,29604,4,1,2.64,4.93,4,143.58,0,23246,1,0,4,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958342,332785,5,219,219,77700,3,1,1.61,0.00,2,118.00,0,16974,1,21,3,0,1,0,0
4958343,332785,5,219,219,88083,3,1,1.95,0.00,3,89.00,0,16974,1,21,3,0,1,0,0
4958344,332785,5,219,219,94508,3,1,1.10,0.00,4,99.00,0,16974,1,21,3,0,1,0,0
4958345,332785,5,219,219,128360,3,1,1.95,0.00,1,139.00,0,16974,1,21,3,0,1,0,1


In [56]:
# Perform logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import roc_auc_score

# Split data
X = df_1.drop('target', axis=1)
y = df_1['target']

splitter = GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 7)
split = splitter.split(X, groups=X['srch_id'])
train_inds, test_inds = next(split)


X_train, X_test, y_train, y_test = X.iloc[train_inds], X.iloc[test_inds], y.iloc[train_inds], y.iloc[test_inds]

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
preds1 = log_reg.predict_proba(X_test)

c:\Users\caspa\Desktop\data-mining-techniques-vu\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [60]:
preds1  

array([[0.96306605, 0.03693395],
       [0.96537174, 0.03462826],
       [0.96381922, 0.03618078],
       ...,
       [0.97866637, 0.02133363],
       [0.9744209 , 0.0255791 ],
       [0.97439504, 0.02560496]])

In [43]:
# No duplicates

# # Get subset of srch_id and prop_id
# df_1 = df_1[['srch_id', 'prop_id', 'target']]

# # count duplicates
# df_1['count'] = df_1.groupby(['srch_id', 'prop_id'])['target'].transform('count')

# df_1.value_counts(['count'])

C:\Users\caspa\AppData\Local\Temp\ipykernel_11256\1035002778.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['count'] = df_1.groupby(['srch_id', 'prop_id'])['target'].transform('count')


count
1        4958347
Name: count, dtype: int64

In [13]:
nunique_list = []
for col in df.columns:
    nunique_list.append(df[col].nunique())
pd.DataFrame({'col': df.columns, 'nunique': nunique_list})

,col,nunique
0,srch_id,199795
1,date_time,198615
2,site_id,34
3,visitor_location_country_id,210
4,visitor_hist_starrating,312
5,visitor_hist_adr_usd,7799
6,prop_country_id,172
7,prop_id,129113
8,prop_starrating,6
9,prop_review_score,10


In [6]:
df['booking_bool'].value_counts()

booking_bool
0    4819957
1     138390
Name: count, dtype: int64

In [14]:
len(df.columns)

54

In [15]:
df.columns

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'